<a href="https://colab.research.google.com/github/alexandergribenchenko/Data_Science_Self_Study/blob/main/Object_Oriented_Programming/Estimators_Sklearn_and_SparkML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01. Sklearn

## 01.01. Train dataset base

In [ ]:
import pandas as pd
  
C1_lst = [-21.57, 1.24, -0.84, 32.25, 0.82, -3.11, 0.46, -18.68, 0.04, 30.87]
C2_lst = [2.49, 2.27, 0.25, -2.33, -2.91, -3.61, 0.58, -2.59, -3.99, 1.54]
C3_lst = [1.14, 4.76, 14.23, -2.65, -3.53, -0.03, 17.50, -0.21, 2.96, 2.31]

df_train = pd.DataFrame(list(zip(C1_lst, C2_lst, C3_lst )), columns =['C1', 'C2', 'C3'])
df_train

,C1,C2,C3
0,-21.57,2.49,1.14
1,1.24,2.27,4.76
2,-0.84,0.25,14.23
3,32.25,-2.33,-2.65
4,0.82,-2.91,-3.53
5,-3.11,-3.61,-0.03
6,0.46,0.58,17.50
7,-18.68,-2.59,-0.21
8,0.04,-3.99,2.96
9,30.87,1.54,2.31


In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   C1      10 non-null     float64
 1   C2      10 non-null     float64
 2   C3      10 non-null     float64
dtypes: float64(3)
memory usage: 368.0 bytes


## 01.02. Function to convert in SKlearn Estimator

In [ ]:
def outliers(df, input_columns):
  
  Q1 = df[input_columns].quantile(0.25, interpolation='lower')
  Q3 = df[input_columns].quantile(0.75, interpolation='lower')
  IQR = Q3-Q1
  outliers_thresholds_down = Q1-1.5*IQR
  outliers_thresholds_up = Q3+1.5*IQR
  
  dict_outliers_thresholds = {col:[outliers_thresholds_down[idx], outliers_thresholds_up[idx]] for idx, col in enumerate(input_columns)}
  
  df_output=df.copy()
  for col in dict_outliers_thresholds.keys():
    df_output[col] = df[col].apply(lambda x: dict_outliers_thresholds[col][0] if x<dict_outliers_thresholds[col][0] else
                                             dict_outliers_thresholds[col][1] if x>dict_outliers_thresholds[col][1] else
                                             x)
  return df_output, dict_outliers_thresholds

In [ ]:
df_output, dict_outliers_thresholds = outliers(df_train, input_columns=['C1','C3'])

In [ ]:
dict_outliers_thresholds

Out[33]: {'C1': [-9.004999999999999, 6.715], 'C3': [-4.965, 7.715]}

In [ ]:
df_output

,C1,C2,C3
0,-9.005,2.49,1.140
1,1.240,2.27,4.760
2,-0.840,0.25,7.715
3,6.715,-2.33,-2.650
4,0.820,-2.91,-3.530
5,-3.110,-3.61,-0.030
6,0.460,0.58,7.715
7,-9.005,-2.59,-0.210
8,0.040,-3.99,2.960
9,6.715,1.54,2.310


In [ ]:
df_train

,C1,C2,C3
0,-21.57,2.49,1.14
1,1.24,2.27,4.76
2,-0.84,0.25,14.23
3,32.25,-2.33,-2.65
4,0.82,-2.91,-3.53
5,-3.11,-3.61,-0.03
6,0.46,0.58,17.50
7,-18.68,-2.59,-0.21
8,0.04,-3.99,2.96
9,30.87,1.54,2.31


## 01.03. SKlearn Estimator

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

In [ ]:
class SKlearnOutliersEstimator(BaseEstimator, TransformerMixin):

  def __init__(self, input_columns=None):
    self.input_columns = input_columns

  def fit(self, X=None, y=None):
    Q1 = X[self.input_columns].quantile(0.25, interpolation='lower')
    Q3 = X[self.input_columns].quantile(0.75, interpolation='lower')
    IQR = Q3-Q1
    outliers_thresholds_down = Q1-1.5*IQR
    outliers_thresholds_up = Q3+1.5*IQR
  
    self.dict_outliers_thresholds_ = {col:[outliers_thresholds_down[idx], outliers_thresholds_up[idx]] for idx, col in enumerate(self.input_columns)}
    return self

  def transform(self, X):
    X_output = X.copy()
    for col in dict_outliers_thresholds.keys():
      X_output[col] = X[col].apply(lambda x: dict_outliers_thresholds[col][0] if x<dict_outliers_thresholds[col][0] else
                                             dict_outliers_thresholds[col][1] if x>dict_outliers_thresholds[col][1] else
                                             x)
    return X_output

In [ ]:
Object_SKlearnOutliersEstimator = SKlearnOutliersEstimator(input_columns=['C1', 'C3'])

In [ ]:
Object_SKlearnOutliersEstimator.fit_transform(X=df_train)

,C1,C2,C3
0,-9.005,2.49,1.140
1,1.240,2.27,4.760
2,-0.840,0.25,7.715
3,6.715,-2.33,-2.650
4,0.820,-2.91,-3.530
5,-3.110,-3.61,-0.030
6,0.460,0.58,7.715
7,-9.005,-2.59,-0.210
8,0.040,-3.99,2.960
9,6.715,1.54,2.310


In [ ]:
Object_SKlearnOutliersEstimator.dict_outliers_thresholds_

Out[47]: {'C1': [-9.004999999999999, 6.715], 'C3': [-4.965, 7.715]}

In [ ]:
df_train

,C1,C2,C3
0,-21.57,2.49,1.14
1,1.24,2.27,4.76
2,-0.84,0.25,14.23
3,32.25,-2.33,-2.65
4,0.82,-2.91,-3.53
5,-3.11,-3.61,-0.03
6,0.46,0.58,17.50
7,-18.68,-2.59,-0.21
8,0.04,-3.99,2.96
9,30.87,1.54,2.31


## 01.04. Test estimator with other dataframe

In [ ]:
C1_lst = [-20, -10, -5, 0, 5, 10, 20]
C2_lst = [-8, -4, -2, 0, 2, 4, 8]
C3_lst = [-8, -4, -2, 0, 2, 4, 8]

df_test = pd.DataFrame(list(zip(C1_lst, C2_lst, C3_lst )), columns =['C1', 'C2', 'C3'])
df_test

,C1,C2,C3
0,-20,-8,-8
1,-10,-4,-4
2,-5,-2,-2
3,0,0,0
4,5,2,2
5,10,4,4
6,20,8,8


In [ ]:
Object_SKlearnOutliersEstimator.dict_outliers_thresholds_

Out[52]: {'C1': [-9.004999999999999, 6.715], 'C3': [-4.965, 7.715]}

In [ ]:
Object_SKlearnOutliersEstimator.transform(df_test)

,C1,C2,C3
0,-9.005,-8,-4.965
1,-9.005,-4,-4.000
2,-5.000,-2,-2.000
3,0.000,0,0.000
4,5.000,2,2.000
5,6.715,4,4.000
6,6.715,8,7.715


# 02. Pyspark (--------Aqui)

## 02.01. Dataset base

In [ ]:
dfs = spark.createDataFrame(df)
dfs.show()

In [ ]:
dfs.printSchema()

## 02.02. Function to convert in Spark.ml Estimator

In [ ]:
dfs.printSchema()

## 02.03. Spark.ml Estimator

In [ ]:
dfs.show()

In [ ]:
from pyspark.ml.param.shared import HasInputCol, HasOutputCol, HasInputCols, HasOutputCols, Param, Params, TypeConverters
from pyspark import keyword_only
import pyspark.sql.functions as F

In [ ]:
class _PySparkOutliersEstimatorParams(HasInputCols):
  
  def __init__(self, *args):
    super().__init__(*args)

In [ ]:
from pyspark.sql.functions import when

In [ ]:
from pyspark.ml import Model

class PySparkOutliersEstimatorModel(Model, _PySparkOutliersEstimatorParams):
  
  # He incluido el parametro _boundary de manera complementaria para traerlo de ExtremeValueCapper
  outlier_thresholds_down_ = Param(Params._dummy(), "outlier_thresholds_down_", "comment_down." "Comment_down")
  outlier_thresholds_up_ = Param(Params._dummy(), "outlier_thresholds_up_", "comment_up." "Comment_up")
  
  @keyword_only
  def __init__(self, inputCols=None, outlier_thresholds_down_=None, outlier_thresholds_up_=None):
    super().__init__()
    kwargs = self._input_kwargs
    self.setParams(**kwargs)
    
  @keyword_only
  def setParams(self, inputCols=None, outlier_thresholds_down_=None, outlier_thresholds_up_=None):
      kwargs = self._input_kwargs
      return self._set(**kwargs)
    
  def _transform(self, dataset):
    input_columns = (self.getInputCols() if self.isSet("inputCols") else self.getInputCols())
    outlier_thresholds_down = self.getOrDefault("outlier_thresholds_down_")
    outlier_thresholds_up = self.getOrDefault("outlier_thresholds_up_")
    
    X_output = dataset
    
    for idx, col in enumerate(input_columns):
      list_from_col = X_output.select(col).rdd.flatMap(lambda x: x).collect()
      lst_not_outliers = [i for i in list_from_col if i>=outlier_thresholds_down[idx] 
                                              and i<=outlier_thresholds_up[idx]]
      min_not_outlier = min(lst_not_outliers)
      max_not_outlier = max(lst_not_outliers)
      
      X_output = X_output.withColumn(col, when(X_output[col] < min_not_outlier, min_not_outlier)
                                         .when(X_output[col] > max_not_outlier, max_not_outlier)
                                         .otherwise(X_output[col]))
      
    return X_output   

In [ ]:
from pyspark.ml import Estimator

class PySparkOutliersEstimator(Estimator, _PySparkOutliersEstimatorParams):
    
  @keyword_only
  def __init__(self, inputCols=None):
    super().__init__()
    kwargs = self._input_kwargs
    self.setParams(**kwargs)
    
  @keyword_only
  def setParams(self, inputCols=None):
      kwargs = self._input_kwargs
      return self._set(**kwargs)
  def setInputCols(self, new_inputCols):
    return self.setParams(inputCols=new_inputCols)
    
  def _fit(self, dataset):
    input_columns = (self.getInputCols() if self.isSet("inputCols") else self.getInputCols())
    Q1 = [dataset.approxQuantile(i, [0.25], 0)[0] for i in input_columns]
    Q3 = [dataset.approxQuantile(i, [0.75], 0)[0] for i in input_columns]
    IQR = [q3_i - q1_i for q3_i, q1_i in zip(Q3,Q1)]
    
    outlier_thresholds_down = [q1_i - 1.5*iqr_i for q1_i, iqr_i in zip(Q1,IQR)]
    outlier_thresholds_up = [q3_i + 1.5*iqr_i for q3_i, iqr_i in zip(Q3,IQR)]
    
#     return outlier_thresholds_down, outlier_thresholds_up

    return PySparkOutliersEstimatorModel(inputCols=input_columns, outlier_thresholds_down_=outlier_thresholds_down, outlier_thresholds_up_=outlier_thresholds_up)

In [ ]:
Object_PySparkOutliersEstimator = PySparkOutliersEstimator(inputCols=['C1','C3'])

In [ ]:
Object_PySparkOutliersEstimator.fit(dfs).transform(dfs).show()

In [ ]:
dfs.show()

In [ ]:
# Evaluamos el tipo de objeto y los parametros previo al entrenamiento
Object_PySparkOutliersEstimator_paramaters = set(dir(Object_PySparkOutliersEstimator))
print(type(Object_PySparkOutliersEstimator))
print(Object_PySparkOutliersEstimator_paramaters)
print(Object_PySparkOutliersEstimator.getOrDefault('inputCols'))

In [ ]:
# Evaluamos el tipo de objeto y los parametros posterior al entrenamiento
Object_PySparkOutliersEstimator_trained = Object_PySparkOutliersEstimator.fit(dfs)
Object_PySparkOutliersEstimator_trained_paramaters = set(dir(Object_PySparkOutliersEstimator_trained))
print(type(Object_PySparkOutliersEstimator_trained))
print(Object_PySparkOutliersEstimator_trained_paramaters)
# print(test_EVC_trained.getOrDefault('_boundary'))
print(Object_PySparkOutliersEstimator_trained.getOrDefault('outlier_thresholds_down_'))
print(Object_PySparkOutliersEstimator_trained.getOrDefault('outlier_thresholds_up_'))

In [ ]:
Object_PySparkOutliersEstimator.fit(dfs).transform(dfs_02).show()

In [ ]:
C1_lst = [-10, -12, -0.84, 32.25, 0.82, -3.11, 0.46, -18.68, 10, 12]
C2_lst = [2.49, 2.27, 0.25, -2.33, -2.91, -3.61, 0.58, -2.59, -3.99, 1.54]
C3_lst = [1.14, 4.76, 14.23, -2.65, -3.53, -0.03, 17.50, -0.21, 2.96, 2.31]


df_02 = pd.DataFrame(list(zip(C1_lst, C2_lst, C3_lst )), columns =['C1', 'C2', 'C3'])
df_02

In [ ]:
dfs_02 = spark.createDataFrame(df_02)
dfs_02.show()